In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import h5py

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=1024)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
train_images_pth = ".\Datasets\camelyonpatch_level_2_split_train_x.h5"
train_labels_pth = ".\Datasets\camelyonpatch_level_2_split_train_y.h5"
test_images_pth = ".\Datasets\camelyonpatch_level_2_split_valid_x.h5"
test_labels_pth = ".\Datasets\camelyonpatch_level_2_split_valid_y.h5"

In [4]:
with h5py.File(train_images_pth,swmr=True) as train_x:
    h5train_images = train_x['x']
    train_images = np.asarray(h5train_images)

with h5py.File(train_labels_pth) as train_y:
    h5train_labels = train_y['y']
    train_labels = np.asarray(h5train_labels)

with h5py.File(test_images_pth) as valid_x:
    h5validation_images = valid_x['x']
    validation_images = np.asarray(h5validation_images)

with h5py.File(test_labels_pth) as valid_y:
    h5validation_labels = valid_y['y']
    validation_labels = np.asarray(h5validation_labels)


In [5]:
print("ANKITH MOTHA - 21BAI1284\n")
print("Dataset Info:")
print("Training Images:",train_images.shape)
print("Training Labels:",train_labels.shape)
print("Validation Images:",validation_images.shape)
print("Validation Labels:",validation_labels.shape)


ANKITH MOTHA - 21BAI1284

Dataset Info:
Training Images: (262144, 96, 96, 3)
Training Labels: (262144, 1, 1, 1)
Validation Images: (32768, 96, 96, 3)
Validation Labels: (32768, 1, 1, 1)


In [6]:
reshaped_train_images = np.asarray([i for i in train_images[:len(train_images)//2]],dtype="float16")
reshaped_validation_images = np.asarray([i for i in validation_images],dtype="float16")
del train_images, validation_images

len_train_images = reshaped_train_images.shape[0]
len_validation_images = reshaped_validation_images.shape[0]

len_train_images,len_validation_images

(131072, 32768)

In [7]:
squeezed_train_labels = np.squeeze(train_labels[:len(train_labels)//2])
squeezed_validation_labels = np.squeeze(validation_labels)

# squeezed_train_labels = train_labels[:len(train_labels)//2]
# squeezed_validation_labels = validation_labels
squeezed_train_labels.shape,squeezed_validation_labels.shape

((131072,), (32768,))

In [8]:
batch_size = 256

# with tf.device("/device:GPU:0"):
#     reshaped_train_images = tf.convert_to_tensor(reshaped_train_images,dtype=tf.float16)
#     reshaped_validation_images = tf.convert_to_tensor(reshaped_validation_images,dtype=tf.float16)
#     squeezed_train_labels = tf.convert_to_tensor(squeezed_train_labels)
#     squeezed_train_labels = tf.convert_to_tensor(squeezed_validation_labels)

# train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# train_datagen.flow(reshaped_train_images, squeezed_train_labels, batch_size=batch_size)
# del reshaped_train_images

# validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
# validation_datagen.flow(reshaped_validation_images, squeezed_validation_labels, batch_size=batch_size)
# del reshaped_validation_images

In [9]:
with tf.device("/device:GPU:0"):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(96,96,3)))
    model.add(tf.keras.layers.Dense(256, activation='relu',))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(0.3))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


    model.compile(optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy'])

    EPOCHS = 15
    history = model.fit(
        reshaped_train_images,squeezed_train_labels,
        steps_per_epoch=len_train_images // batch_size,
        epochs=EPOCHS,
        validation_data=(reshaped_validation_images,squeezed_validation_labels),
        validation_steps=len_validation_images// batch_size
    )

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

## CAT and DOG Classification - 21BAI1284

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
tf.random.set_seed(42)

In [ ]:
base_dir = ".\Datasets\Cats_Dogs_Kaggle"
train_dir = os.path.join(base_dir, 'training_set')
validation_dir = os.path.join(base_dir, 'test_set')

train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

#Understanding the data
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

In [ ]:
IMG_SHAPE  = 150
BATCH_SIZE = 32

train_image_generator      = ImageDataGenerator(rescale=1./255)
validation_image_generator = ImageDataGenerator(rescale=1./255)

train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                        batch_size = BATCH_SIZE,
                                                        shuffle=True,
                                                        target_size=(IMG_SHAPE,IMG_SHAPE),
                                                        class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                        batch_size = BATCH_SIZE,
                                                        shuffle=False,
                                                        target_size=(IMG_SHAPE,IMG_SHAPE),
                                                        class_mode='binary')

sample_training_images, sample_training_labels = next(train_data_gen)
label_names = ["CAT","DOG"]

def plotImages(images_arr,labels):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, labs, ax in zip(images_arr,labels, axes):
        ax.imshow(img)
        ax.text(0.5,-0.2, str(label_names[int(labs)])+"\n21BAI1284", size=12, ha="center",transform=ax.transAxes)
    plt.tight_layout()
    plt.show()
plotImages(sample_training_images[:5],sample_training_labels[:5])

print(sample_training_images[0])

In [ ]:

# ANKITH MOTHA - 21BAI1284

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(IMG_SHAPE,IMG_SHAPE, 3)))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'])

EPOCHS = 25
history = model.fit(
    train_data_gen,
    steps_per_epoch=total_train // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=total_val// BATCH_SIZE
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('ANKITH - Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('ANKITH - Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()